![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column       | Description                                  |
|------------- |--------------------------------------------- |
| `student_id`   | A unique ID for each student.                 |
| `city`  | A code for the city the student lives in.  |
| `city_development_index` | A scaled development index for the city.       |
| `gender` | The student's gender.       |
| `relevant_experience` | An indicator of the student's work relevant experience.       |
| `enrolled_university` | The type of university course enrolled in (if any).       |
| `education_level` | The student's education level.       |
| `major_discipline` | The educational discipline of the student.       |
| `experience` | The student's total work experience (in years).       |
| `company_size` | The number of employees at the student's current employer.       |
| `last_new_job` | The number of years between the student's current and previous jobs.       |
| `training_hours` | The number of hours of training completed.       |
| `job_change` | An indicator of whether the student is looking for a new job (`1`) or not (`0`).       |

In [309]:
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv('customer_train.csv')

In [310]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [311]:
ds_jobs.experience.unique()

array(['>20', '15', '5', '<1', '11', '13', '7', '17', '2', '16', '1', '4',
       '10', '14', '18', '19', '12', '3', '6', '9', '8', '20', nan],
      dtype=object)

In [312]:
ds_jobs['job_change'].unique()

array([1, 0])

In [313]:
# Filter the dataset
ds_jobs_clean = ds_jobs[(ds_jobs['experience'].isin(['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'])) & (ds_jobs['company_size'].isin(['1000-4999', '5000-9999', '10000+']))]

# Convert columns to the specified data types
ds_jobs_clean = ds_jobs_clean.astype({
    'student_id': 'int32',
    'city': 'category',
    'city_development_index': 'float16',
    'gender': 'category',
    'relevant_experience': 'category',
    'enrolled_university': 'category',
    'education_level': 'category',
    'major_discipline': 'category',
    'experience': 'category',
    'company_size': 'category',
    'company_type': 'category',
    'last_new_job': 'category',
    'training_hours': 'int32',
    'job_change': 'int32'
})

# Set the order of ordinal categorical data
ordinal_categories_edu = ['Primary School', 'High School', 'Graduate', 'Masters', 'Phd']
ordinal_categories_comp = ['<10','10-49', '50-99', '100-499', '500-999', '1000-4999', '5000-9999', '10000+']
ordinal_categories_rel_exp = ['No relevant experience', 'Has relevant experience']
ordinal_categories_enr_uni = ['no_enrollment', 'Part time course', 'Full time course']
ordinal_categories_exp = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
ordinal_categories_last_new = ['never', '1', '2', '3','4', '>4']

# Apply the ordering
ds_jobs_clean['education_level'] = pd.Categorical(ds_jobs_clean['education_level'], categories=ordinal_categories_edu, ordered=True)
ds_jobs_clean['company_size'] = pd.Categorical(ds_jobs_clean['company_size'], categories=ordinal_categories_comp, ordered=True)
ds_jobs_clean['relevant_experience'] = pd.Categorical(ds_jobs_clean['relevant_experience'], categories=ordinal_categories_rel_exp, ordered=True)
ds_jobs_clean['enrolled_university'] = pd.Categorical(ds_jobs_clean['enrolled_university'], categories=ordinal_categories_enr_uni, ordered=True)
ds_jobs_clean['experience'] = pd.Categorical(ds_jobs_clean['experience'], categories=ordinal_categories_exp, ordered=True)
ds_jobs_clean['last_new_job'] = pd.Categorical(ds_jobs_clean['last_new_job'], categories=ordinal_categories_last_new, ordered=True)

# Ensure the columns are in the same order as the original dataset
ds_jobs_clean = ds_jobs_clean[ds_jobs.columns]

# Print the first 5 rows of the cleaned dataset
print(ds_jobs_clean.head())

    student_id      city  ...  training_hours job_change
9          699  city_103  ...             123          0
12       25619   city_61  ...              23          0
31       22293  city_103  ...             141          0
34       26494   city_16  ...             145          0
40        2547  city_114  ...              14          0

[5 rows x 14 columns]


In [314]:
ds_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  int64  
dtypes: float64(1), int64(3), object(10)
me

In [315]:
ds_jobs_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2201 entries, 9 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              2201 non-null   int32   
 1   city                    2201 non-null   category
 2   city_development_index  2201 non-null   float16 
 3   gender                  1821 non-null   category
 4   relevant_experience     2201 non-null   category
 5   enrolled_university     2185 non-null   category
 6   education_level         2184 non-null   category
 7   major_discipline        2097 non-null   category
 8   experience              0 non-null      category
 9   company_size            2201 non-null   category
 10  company_type            2144 non-null   category
 11  last_new_job            2184 non-null   category
 12  training_hours          2201 non-null   int32   
 13  job_change              2201 non-null   int32   
dtypes: category(10), float1